# MNIST trials

In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [5]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

from tensorflow.examples.tutorials.mnist import input_data

DATA_PATH = '/home/prathamesh/undergrad/btech_project/misc/mnist_data'

mnist_dat = input_data.read_data_sets(DATA_PATH, one_hot=True);

Extracting /home/prathamesh/undergrad/btech_project/misc/mnist_data/train-images-idx3-ubyte.gz
Extracting /home/prathamesh/undergrad/btech_project/misc/mnist_data/train-labels-idx1-ubyte.gz
Extracting /home/prathamesh/undergrad/btech_project/misc/mnist_data/t10k-images-idx3-ubyte.gz
Extracting /home/prathamesh/undergrad/btech_project/misc/mnist_data/t10k-labels-idx1-ubyte.gz


In [48]:

def max_pool(input_, kernel=[1,2,2,1], stride=[1,2,2,1], padd='SAME'):
    return tf.nn.max_pool(input_, ksize=kernel, strides=stride, padding=padd)

def get_weights(size):
    return tf.Variable(tf.random_normal(shape=size, stddev=0.4))

def get_bias(size):
    return tf.Variable(tf.constant(0.1, shape=size))

def full_conn(inp, size, d1):
    W = get_weights([d1,size])
    bias = get_bias([size]);
    return tf.matmul(inp, W) + bias

def conv_layer(input_, filtr, stride=[1,1,1,1], padd='SAME'):
    W = get_weights(filtr);
    bias = get_bias([filtr[3]])
    return tf.nn.relu(tf.nn.conv2d(input_, W, strides=stride, padding=padd) + bias)


In [92]:
#import helpers as h

with tf.Graph().as_default():
    
    x = tf.placeholder(tf.float32, shape=[None,784]);
    x_im = tf.reshape(x, [-1,28,28,1])
    y_true = tf.placeholder(tf.float32, shape=[None,10])
    keep_prob = tf.placeholder(tf.float32)
    
    conv1 = conv_layer(x_im, [5,5,1,32]);
    conv1 = max_pool(conv1)
    
    conv2 = conv_layer(conv1, [5,5,32,64])
    conv2 = max_pool(conv2)
    
    conv_flat = tf.reshape(conv2, [-1, 7*7*64])
    dense1 = tf.nn.relu(full_conn(conv_flat, size=1024, d1=7*7*64))
    d1_drop = tf.nn.dropout(dense1, keep_prob=keep_prob)
    
    y_pred = full_conn(d1_drop, size=10, d1=1024)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
    corr_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
    accuracy = tf.reduce_mean(tf.cast(corr_pred, tf.float32))
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #ss = s.run(y_pred, feed_dict={x: np.random.randn(5,784), y_true: np.random.randn(5,10), keep_prob: 0.5})
        EPOCHS = 2

        for epoch in range(EPOCHS):
            for step in range(500):
                batch = mnist_dat.train.next_batch(100)
                if step%100 == 0:
                    ssd = sess.run(accuracy,feed_dict={x:batch[0], y_true:batch[1],keep_prob:0.5})
                    print("Epoch {} Accuracy = {} at step {}".format(epoch+1,ssd,step+1))
                sess.run(optimizer, feed_dict={x: batch[0], y_true: batch[1], keep_prob:0.5})
                
        X = mnist_dat.test.images.reshape(10,1000,784)
        Y = mnist_dat.test.labels.reshape(10,1000,10)
        acc = []
        for i in range(10):
            acc.append(sess.run(accuracy, feed_dict={x: X[i],y_true:Y[i], keep_prob:0.5}))
        t_acc = np.mean(acc)
        print("Test accuracy: {0}".format(t_acc))

Epoch 1 Accuracy = 0.05999999865889549 at step 1
Epoch 1 Accuracy = 0.800000011920929 at step 101
Epoch 1 Accuracy = 0.8600000143051147 at step 201
Epoch 1 Accuracy = 0.8700000047683716 at step 301
Epoch 1 Accuracy = 0.9399999976158142 at step 401
Epoch 2 Accuracy = 0.9300000071525574 at step 1
Epoch 2 Accuracy = 0.9100000262260437 at step 101
Epoch 2 Accuracy = 0.8999999761581421 at step 201
Epoch 2 Accuracy = 0.949999988079071 at step 301
Epoch 2 Accuracy = 0.8700000047683716 at step 401
Test accuracy: 0.9318000078201294
